# Matching litigation firm with others

### Litigation
- nameprocessing 필요 => 업그레이드 한 번 하자 (corporate, incorporated, company도 제거 좀...)
- compustat, assignee matching 성공한 기업들 데이터만 필요
- raw_assignee가 제일 잘 붙을거임

### matching 방법
- raw_assignee와 이름 기준으로 matching 하자
- nameprocessing 결과로 1차 matching
- raw_assignee -> assignee -> GVkey 순서로 match
- 나머지에 대해 manual matching

### 마무리
- patent_lit_key.csv

### 1. nameprocessing & 1차 matching

In [1]:
#-*- coding: utf-8 -*-
import os

os.chdir('E:/apps')

### AG, BV, CORP, CO, kabushiki kaisha 처리 필요

def nameprocessing(s):
    s = s.lower()
    s = s.replace("-"," ")
    s = s.replace("&",'and')

    rm = ['.',',',';',"'",'kabushiki kaisha','kabushikikaisha','limited','company','corporation','incorporated']

    for i in rm:
        s = s.replace(i,"")

    sp=s.split()
    sp.reverse()

    abb=['inc','ltd','llc','pte','plc','kft','co','gmbh','corp','ag','bv','kg']

    for i in range(0,len(sp)):
        if sp[i] in abb:
            sp[i]=''
    
    sp.reverse()
    return ''.join(sp)

In [2]:
s="kabushiki kaisha Seven-Eleven' Japancorporat'ion Co., Ltd. company cor;poRati'On kabushiki kaisha co & KG"

nameprocessing(s)

'sevenelevenjapanand'

In [3]:
import pandas as pd

df = pd.read_csv('assignee_raw.csv')
df['new_raw_assignee'] = df['raw_assignee'].apply(nameprocessing)
del df['assignee']
df.drop_duplicates(inplace=True)

df1 = pd.read_csv('litigation/lit_name.csv')
df1['new_raw_assignee'] = df1['name'].apply(nameprocessing)
df1.drop_duplicates(inplace=True)

df = df.merge(df1, on='new_raw_assignee', how='left', sort=True)
df.drop_duplicates(inplace=True)

df.to_csv('pat_lit_raw_match.csv',index=False)
df1.to_csv('lit_new.csv',index=False)

### 2. raw_assignee -> assignee -> GVkey 순서로 match

In [5]:
import os
import pandas as pd

os.chdir('E:/apps')

df = pd.read_csv('pat_lit_raw_match.csv')
df1 = pd.read_csv('assignee_raw.csv')

del df['new_raw_assignee']
df1 = df1.merge(df, on='raw_assignee', how='left', sort=True)
df1.drop_duplicates(inplace=True)

del df, df1['raw_assignee']
df2 = pd.read_csv('assignee_gvkey.csv')
df2 = df2.merge(df1, on='assignee', how='left', sort=True)
df2.drop_duplicates(inplace=True)

df2.to_csv('patent_lit_key.csv',index=False)

### 3. matching 성공률 확인

In [11]:
df1 = pd.read_csv('litigation/patent_names.csv')
df1 = df1[['case_row_id','name']]
df1.drop_duplicates(inplace=True)

df2 = pd.read_csv('patent_lit_key.csv')
df2 = df2['name'].to_frame()
df2.drop_duplicates(inplace=True)

df2 = df2.merge(df1, on='name', how='left', sort=True)
df2.drop_duplicates(inplace=True)

In [14]:
cases = df2['case_row_id'].to_frame()
cases.drop_duplicates(inplace=True)
len(cases)

9457

In [16]:
cases.to_csv('related_cases.csv',index=False)